#Нейро-тьютор v4.7

In [ ]:
#@title Конфигурация НС { display-mode: "form" }

l = '**********Системные настройки*********' #@param ['**********Системные настройки*********']
# Путь к файлу с сылками на лекции

pkl_url = 'https://drive.google.com/file/d/1-0mfXxZkUhEWIYq7cDXPgWikdnZ3tGOn/view?usp=drive_link' #@param {type:"string"}
faiss_url = 'https://drive.google.com/file/d/1-DOm6s5Rp_qYi0rrotTVVc9ec0R-BkTC/view?usp=drive_link' #@param {type:"string"}
faiss_link = '/content/faiss_data'
#video_list_patch = '/content/drive/MyDrive/MGPU/Лекции.xlsx' #@param {type:"string", description: "Путь к файлу с сылками на лекции"}
# Путь к папке на Google Диске
folder_path = "/content/drive/My Drive/MGPU/audio/"
# Укажите путь к файлу индекса
#index_path = "/content/drive/MyDrive/MGPU/audio/faiss_index_all.bin" #@param {type:"string"}
# Размер чанков в токенах для формирования базы знаний
pcs_token = 500

l = '*********Конфигурация GPT*********' #@param ['*********Конфигурация GPT*********']
# Конфигурация GPT
# модель GPT
model_gpt = 'gpt-4o-mini' #@param ['gpt-4o-mini', 'gpt-4o', 'gpt-4']
# Температура
temperature = 0.1  # @param {type: "slider", min: 0.0, max: 1, step: 0.1}
# Промт для формирования экзаменационного вопроса
system = 'Вы лучший преподаватель Психолого-педагогического института. Ответьте развернуто на вопрос, на основе отрывков из лекций. При формировании ответа на лекции ссылаться не надо.' # @param {type:"string"}
# Количество чанков, передаваемых в модель для формирования ответа
k_chunks = 3 # @param {type: "integer"}

# Конфигурация Faster Whisper
l = '*********Конфигурация Faster Whisper*********' #@param ['*********Конфигурация Faster Whisper*********']
model_whisper = "small" #@param ["small", "medium", "large-v3"]
# Параметр с условием выбора GPU
use_gpu = False  # @param {type: "boolean"}

# Количество чанков объединяющихся для пересказа
group_chunk = 4

In [ ]:
#@title Установка и импорт библиотек { display-mode: "form" }
!pip install -q requests beautifulsoup4 ffmpeg requests-html nest_asyncio pydub SpeechRecognition lxml_html_clean requests-html
#!pip install --upgrade requests-html google-genai
!pip install -q faster-whisper==1.0.3 ctranslate2==4.4.0
!pip install -q yt-dlp -U
!pip install -q pyaspeller
!pip install -q --upgrade langchain_openai faiss-cpu openai tiktoken langchain_community langchain
!pip install -q langchain-openai -U
!pip install -q https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.6.0/ru_core_news_sm-3.6.0.tar.gz

# Импорт модулей и библиотек
import nest_asyncio
nest_asyncio.apply()
import re
import requests
from pydub import AudioSegment
import yt_dlp
import pandas as pd
from requests_html import AsyncHTMLSession
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from faster_whisper import WhisperModel
from pyaspeller import YandexSpeller
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display
import asyncio
import shutil
import time
import subprocess
import numpy as np
import io
import tempfile
import tiktoken
from scipy.spatial.distance import cdist
import json
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    MarkdownHeaderTextSplitter
)
from bs4 import BeautifulSoup
from langchain.vectorstores import FAISS
import openai
from openai import OpenAI
from langchain.docstore.document import Document as Doc
import matplotlib.pyplot as plt
from google.colab import userdata
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

# Получение ключа GPT
openai_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key


#@title Загрузка модели Faster Whisper
if use_gpu:
    device = "cuda"
else:
    device = "cpu"
print(f"Используется устройство: {device}")

model = WhisperModel(model_whisper, device = device, compute_type="int8")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.4.0 requires websockets<15.0dev,>=13.0, but you have websockets 10.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

In [ ]:
#@title Функции по распознаванию аудиолекций: { display-mode: "form" }

# Парсинг Dzen по ссылке video/watch и получение ссылки dzen.ru/embed/
async def get_dzen_embed (url):
    # Применяем nest_asyncio
    nest_asyncio.apply()

    # Создаем асинхронную сессию
    session = AsyncHTMLSession()

    # Заголовки для имитации запроса от браузера
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    async def fetch_video():
        try:
            # Открываем страницу с заголовками
            response = await session.get(url, headers=headers)

            # Ждем, чтобы JavaScript выполнился (если это необходимо)
            await response.html.arender(sleep=10, timeout=60)  # Увеличьте время ожидания

            # Получаем HTML-код страницы
            html_content = response.html.html

            # Используем регулярное выражение для поиска ссылки dzen.ru/embed/
            embed_link = re.search(r'https://dzen\.ru/embed/[^\s"]+', html_content)

            if embed_link:
                return embed_link.group(0)  # Возвращаем найденную ссылку
            else:
                print("Ссылка не найдена.")
                return None  # Возвращаем None, если ссылка не найдена

        except Exception as e:
            print(f'Произошла ошибка: {e}')
            return None  # Возвращаем None в случае ошибки

    # Запускаем асинхронную функцию
    embed_url = await fetch_video()
    print('Ссылка dzen', embed_url)  # Выводим найденную ссылку
    return embed_url

# Очищение название файла от недопустимых символов
def sanitize_filename(filename):
    sanitized = re.sub(r'[\\/*?:"<>|]', '', filename)  # Удаляем запрещенные символы
    sanitized = sanitized.replace('\n', ' ').replace('\r', ' ')  # Удаляем переводы строк
    sanitized = sanitized.strip()  # Удаляем лишние пробелы по краям
    return sanitized

# Получение информации о видео
def get_video_info(embed_url):
    # Отправляем GET-запрос к странице embed
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    response = requests.get(embed_url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch the page. Status code: {response.status_code}")

    # Устанавливаем кодировку UTF-8
    response.encoding = 'utf-8'
    html_content = response.text

    # Парсим HTML-код страницы
    soup = BeautifulSoup(html_content, 'html.parser')

    # Находим скрипт с JSON-данными
    script_tag = soup.find('script', string=re.compile(r'Dzen\.player\.init'))
    if not script_tag:
        raise Exception("Script tag with video data not found.")

    # Извлекаем JSON-объект из содержимого скрипта
    match = re.search(r'Dzen\.player\.init\((\{.*\})\)', script_tag.string)
    if not match:
        raise Exception("JSON data not found in script tag.")

    try:
        data = json.loads(match.group(1))
    except json.JSONDecodeError as e:
        raise Exception(f"Failed to decode JSON: {e}")

    # Ищем ссылку на HLS-поток и заголовок видео
    streams = data.get('data', {}).get('content', {}).get('streams', [])
    hls_stream = next((stream['url'] for stream in streams if stream.get('type') == 'HLS'), None)
    title = data.get('data', {}).get('content', {}).get('title', 'Нет заголовка')

    if not hls_stream:
        raise Exception("HLS video URL not found in the data.")

    # Очищаем заголовок для использования в качестве имени файла
    sanitized_title = sanitize_filename(title)

    return {
        "video_url": hls_stream,
        "title": title,  # Исходный заголовок
        "sanitized_title": sanitized_title  # Очищенный заголовок
    }

def download_audio(video_info):
    """
    Скачивает аудио из видео с помощью ffmpeg.
    """
    video_url = video_info["video_url"]
    sanitized_title = video_info["sanitized_title"]
    output_file = f"{sanitized_title}.wav"  # Имя выходного файла

    # Проверяем, существует ли уже файл
    if os.path.exists(output_file):
        print(f"Файл '{output_file}' уже существует. Пропускаем скачивание.")
        return

    # Команда ffmpeg для скачивания аудио
    command = [
        "ffmpeg",
        "-loglevel", "quiet",  # Отключаем вывод логов
        "-headers", "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "-i", video_url,  # Входной URL
        "-vn",  # Отключаем видео
        "-c:a", "pcm_s16le",  # Кодек аудио
        "-ar", "44100",  # Частота дискретизации
        output_file  # Выходной файл
    ]

    try:
        print(f"Скачивание аудио: {output_file}")
        subprocess.run(command, check=True)  # Запускаем ffmpeg
        print(f"Аудио успешно сохранено как: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при скачивании аудио: {e}")

# Траснкрибация аудиодорожек
def trancribe_audio (lecture_title):
    # Загрузка аудиофайла
    audio_file_path = f"{lecture_title}.wav"
    audio = AudioSegment.from_file(audio_file_path)

    # Определяем длину сегмента (5 минут)
    segment_length = 5 * 60 * 1000  # 5 минут в миллисекундах

    # Разбиваем аудиофайл на сегменты
    segments = []
    for i in tqdm(range(0, len(audio), segment_length), desc=f"Экспорт сегментов {lecture_title}"):
        segment = audio[i:i + segment_length]
        segment_file_path = f"/content/segment_{i // segment_length}.wav"
        segment.export(segment_file_path, format="wav")
        segments.append((segment_file_path, i))  # Сохраняем путь к файлу и начальное время

    # Запускаем распознавание текста для каждого сегмента
    segments_segments = []
    for segment_file, start_time in tqdm(segments, desc="Распознавание текста"):
        segment_segments, info = model.transcribe(segment_file, beam_size=5)

        print(f"Обнаруженный язык '{info.language}' с вероятностью {round(info.language_probability, 2) * 100} %\n\n")

        segment_segments = list(segment_segments)  # Транскрипция будет доступна здесь

        # Вывод с временными отметками
        for segment in segment_segments:
            # Добавляем начальное время к временным меткам
            adjusted_start = segment.start + start_time / 1000  # Преобразуем миллисекунды в секунды
            adjusted_end = segment.end + start_time / 1000
            print(f"[{adjusted_start:.2f}s -> {adjusted_end:.2f}s] {segment.text}")

        segments_segments.extend(segment_segments)

    # Удаляем временные сегменты, если они больше не нужны
    for segment_file, _ in tqdm(segments, desc="Удаление временных файлов"):
        os.remove(segment_file)

    text_segments = list(segments_segments)  # Транскрипция будет доступна здесь.


    # Сохраняем рассшифровку в текстовый файл
    file_txt_path = f'/content/drive/My Drive/MGPU/audio/{lecture_title}_transcript_{model_whisper}.txt'
    with open(file_txt_path, 'w') as file:
        for i in range(len(text_segments)):
            file.write(text_segments[i].text.strip() + '\n')
    print(f'Текстовый файл сохранен', file_txt_path)

    return file_txt_path

# Скачивание аудио дорожки с VK - убираем недопустимые символы
def process_vk(url):
    # Функция для очистки имени файла от недопустимых символов
    def sanitize_filename(filename):
        return re.sub(r'[\\/*?:"<>|]', '', filename).replace('#', '')  # Удаляем недопустимые символы

    # Параметры для скачивания только аудиодорожки
    ydl_opts = {
        'format': 'bestaudio/best',  # Скачиваем лучшую аудиодорожку
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',  # Используем FFmpeg для извлечения аудио
            'preferredcodec': 'mp3',      # Формат аудио
            'preferredquality': '192',     # Качество аудио
        }],
        'outtmpl': '%(title)s.%(ext)s',  # Шаблон имени выходного файла
    }

    # Скачивание аудиодорожки
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)  # Получаем информацию о видео без скачивания

        # Очищаем название файла
        sanitized_title = sanitize_filename(info_dict.get('title', 'unknown'))

        # Обновляем шаблон имени файла с очищенным названием
        ydl.params['outtmpl'] = {'default': f"/content/{sanitized_title}.%(ext)s"}

        # Теперь скачиваем файл с новым именем
        ydl.download([url])

        # Получаем метаданные
        lecture_title = sanitized_title
        artist = info_dict.get('uploader', None)
        duration = info_dict.get('duration', None)

        print(f'Скачанный файл: {lecture_title}.mp3')  # Выводим имя файла
        print(f'Название: {lecture_title}')
        print(f'Исполнитель: {artist}')
        print(f'Длительность: {duration} секунд')

    # Форматируем имя файла для сохранения
    file_name_mp3 = f"/content/{lecture_title}.mp3"

    # Конвертация audio.mp3 в audio.wav
    print(f"Обработка файла: {file_name_mp3}")
    audio = AudioSegment.from_file(file_name_mp3)
    audio.export(f"/content/{lecture_title}.wav", format="wav")

    return lecture_title


# Функция-отработка НОВОЙ загрузки DZEN
async def get_links3(extracted_links):
    file_list = []

    # Проверяем, является ли входной параметр списком или строкой
    if isinstance(extracted_links, str):  # Если передана одна ссылка (строка)
        link = extracted_links
        print(link)
        if "dzen.ru" in link:
            print(f"Обрабатываю Dzen ссылку: {link}")
            embed_url = await get_dzen_embed (link)
            print (embed_url)
            try:
                video_info = get_video_info(embed_url)
                print("Ссылка на видео:", video_info["video_url"])
                print("Исходный заголовок:", video_info["title"])
                print("Очищенный заголовок (для файла):", video_info["sanitized_title"])

                # Скачиваем аудио
                download_audio(video_info)
            except Exception as e:
                print("Ошибка:", str(e))

            print(f'Идет распознавание лекции {video_info["sanitized_title"]}...')
            file_txt_path = trancribe_audio(video_info["sanitized_title"])
            lecture_name = video_info["sanitized_title"]
            file_list.append(file_txt_path)
        elif "vk.com" in link:
            print(f"Обрабатываю VK ссылку: {link}")
            lecture_title = process_vk(link)  # Вызов функции для VK
            print(f'Идет распознавание лекции {lecture_title}...')
            file_txt_path = trancribe_audio(lecture_title)
            lecture_name = lecture_title
            file_list.append(file_txt_path)
        else:
            print(f"Неизвестный тип ссылки: {link}")
    elif isinstance(extracted_links, list):  # Если передан список ссылок
        for link in extracted_links:
            print(link)
            if "dzen.ru" in link:
                print(f"Обрабатываю Dzen ссылку: {link}")
                embed_url = await get_dzen_embed (link)
                print (embed_url)
                try:
                    video_info = get_video_info(embed_url)
                    print("Ссылка на видео:", video_info["video_url"])
                    print("Исходный заголовок:", video_info["title"])
                    print("Очищенный заголовок (для файла):", video_info["sanitized_title"])

                    # Скачиваем аудио
                    download_audio(video_info)
                except Exception as e:
                    print("Ошибка:", str(e))

                print(f'Идет распознавание лекции {video_info["sanitized_title"]}...')
                file_txt_path = trancribe_audio(video_info["sanitized_title"])
                lecture_name = video_info["sanitized_title"]
                file_list.append(file_txt_path)
            elif "vk.com" in link:
                print(f"Обрабатываю VK ссылку: {link}")
                lecture_title = process_vk(link)  # Вызов функции для VK
                print(f'Идет распознавание лекции {lecture_title}...')
                file_txt_path = trancribe_audio(lecture_title)
                lecture_name = lecture_title
                file_list.append(file_txt_path)
            else:
                print(f"Неизвестный тип ссылки: {link}")
    else:
        print("Неподдерживаемый формат входных данных. Ожидался str или list.")
        return []

    return file_list, lecture_name


In [ ]:
#@title Функции по формированию базы знаний, а также модели GPT { display-mode: "form" }

# Загрузка базы знаний
def faiss_gdown(faiss_link=faiss_link, pkl_url=pkl_url, faiss_url=faiss_url):
    try:
        # Создание директории
        os.makedirs(faiss_link, exist_ok=True)
        print(f"Директория создана: {faiss_link}")

        # Определение путей к файлам
        pkl_path = f'{faiss_link}/index.pkl'
        faiss_path = f'{faiss_link}/index.faiss'

        # Проверка наличия файлов
        if not os.path.exists(pkl_path) or not os.path.exists(faiss_path):
            print("Файлы не найдены. Начинаю загрузку...")

            # Скачивание файлов
            !gdown --fuzzy {pkl_url} -O {pkl_path}
            !gdown --fuzzy {faiss_url} -O {faiss_path}

            print("Файлы успешно загружены.")
        else:
            print("Файлы уже существуют. Пропускаю загрузку.")

        # Загрузка индекса
        db = FAISS.load_local(
            folder_path=faiss_link,
            embeddings=OpenAIEmbeddings(),
            allow_dangerous_deserialization=True
        )
        print("База знаний успешно загружена.")

        return db

    except Exception as e:
        print(f"Произошла ошибка: {e}")
        return None

# Функция для загрузки документа по сслыке и в текстовом формате
def load_doc_id_text(file_path):
    # Open the document and read it as plain text
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Функция возвращает количество токенов в строке
def num_tokens_from_string(string: str, encoding_name: str) -> int:
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

# Функция по формированию чанков базы знаний
def split_text(text, max_count):
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
        ("#####", "Header 5"),
        ("######", "Header 6"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    fragments = markdown_splitter.split_text(text)

    # Подсчет токенов для каждого фрагмента
    fragment_token_counts = [num_tokens_from_string(fragment.page_content, "cl100k_base") for fragment in fragments]


    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_count,
        chunk_overlap=0,
        length_function=lambda x: num_tokens_from_string(x, "cl100k_base")
    )

    source_chunks = [
        Doc(page_content=chunk, metadata=fragment.metadata)
        for fragment in fragments
        for chunk in splitter.split_text(fragment.page_content)
    ]

    # Подсчет токенов для каждого source_chunk
    source_chunk_token_counts = [num_tokens_from_string(chunk.page_content, "cl100k_base") for chunk in source_chunks]


    return source_chunks, fragments

#@title Создание эмбеддинга базы лекции/ий
def embed_bd (data_from_markdown, folder_path):
    source_chunks, fragments = split_text(data_from_markdown, pcs_token)

    # Инициализирум модель эмбеддингов
    embeddings = OpenAIEmbeddings()

    # Создадим индексную базу из разделенных фрагментов текста
    db = FAISS.from_documents(source_chunks, embeddings)

    lesson_db = f"{folder_path}faiss_index_lesson.bin"

    # Сохраним индекс в файл
    db.save_local(lesson_db)

    print (f'База создана в {lesson_db}')

    return lesson_db

# Объединение чанков на группы
def split_into_groups(chunks, group_size=group_chunk):
    for i in range(0, len(chunks), group_size):
        yield chunks[i:i + group_size]

# Модель по формированию ответов на основе отрывков лекций
def answer_index(system, topic, search_index, model_gpt, temperature, k_chunks, verbose=0):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=k_chunks)
    if verbose: print('\n ===========================================: ')
    #message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n =======\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n =======\nОтрывок лекции №{i+1}\n=====' + str(doc.metadata) + '\n================' + str(doc.page_content) + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model=model_gpt,
        messages=messages,
        temperature=temperature
    )
    answer = completion.choices[0].message.content
    return message_content, answer  # возвращает ответ


#@title !Функции для перессказа!
nlp = spacy.load("ru_core_news_sm")

def count_tokens(text):
    """Подсчет количества токенов в тексте."""
    return len(nlp(text))

def get_relevant_sentences(text, max_sentences=5):
    """
    Выбирает наиболее релевантные предложения из текста на основе TF-IDF,
    сохраняя их исходный порядок.
    Возвращает список предложений.
    """
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]

    print(f"Количество предложений в исходном тексте: {len(sentences)}")

    if len(sentences) <= max_sentences:
        return sentences  # Если предложений мало, возвращаем все

    # Вычисляем TF-IDF для каждого предложения
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    sentence_scores = tfidf_matrix.sum(axis=1)

    # Создаем список кортежей (индекс, важность)
    scored_sentences = [(idx, score[0]) for idx, score in enumerate(sentence_scores)]

    # Сортируем предложения по важности, но сохраняем индексы
    ranked_sentences = sorted(scored_sentences, key=lambda x: x[1], reverse=True)

    # Выбираем наиболее релевантные индексы
    relevant_indices = [idx for idx, _ in ranked_sentences[:max_sentences - 4]]  # Исключаем 2 первых и 2 последних


    # Добавляем  первое и последнее предложения
    #relevant_indices = [0] + relevant_indices + [len(sentences) - 1]

    # Добавляем 2 первых и 2 последних предложения
    relevant_indices = list(range(0, 2)) + relevant_indices + list(range(len(sentences) - 2, len(sentences)))

    # Сортируем индексы, чтобы сохранить исходный порядок
    relevant_indices = sorted(relevant_indices)

    return [sentences[i] for i in relevant_indices]


def rewrite_reduced_text(model_gpt, reduced_group, temperature, lesson_title):
    client = OpenAI()
    messages = [
        {"role": "system", "content": f'Перескажи отрывки лекции: {lesson_title}'},
        {"role": "user", "content": f'{reduced_group}'}
    ]
    response = client.chat.completions.create(
        model=model_gpt,
        messages=messages,
        max_tokens=2000,
        temperature=temperature
    )
    return response.choices[0].message.content

In [ ]:
#@title Форма для добавления лекции в базу знаний { display-mode: "form" }

# Область вывода служебной информации
output_info = widgets.Output(layout={'border': '1px solid black', 'width': '70%', 'height': '300px', 'overflow_y': 'scroll'})

# Загрузка базы знаний
with output_info:
    db = faiss_gdown()
    if db is not None:
        print("База знаний готова к использованию!")
        print("Добавьте лекцию в базу знаний!")
    else:
        print("Не удалось загрузить базу знаний.")
        raise ValueError("Прерываю выполнение, так как база знаний не загружена.")

# Создание виджетов для ввода данных
discipline_input = widgets.Text(description="Дисциплина:", placeholder="Введите название дисциплины")
topic_input = widgets.Text(description="Тема лекции:", placeholder="Введите тему лекции")
link_input = widgets.Text(description="Ссылка на лекцию:", placeholder="Введите ссылку на лекцию")
add_button = widgets.Button(description="Добавить лекцию", button_style='success')

# Асинхронная функция для обработки логики
async def process_lecture(discipline, topic, link):
    try:
        with output_info:
            output_info.clear_output()  # Очистка предыдущих сообщений
            print(f"Начинаю обработку лекции:\nДисциплина: {discipline}\nТема: {topic}\nСсылка: {link}")

            # Вызов асинхронной функции get_links3
            file_txt_patch, lecture_name = await get_links3(link)
            knowledge_base = ""
            print(f"Получен путь к файлу: {file_txt_patch}")

            # Загрузка текста лекции
            file_path = file_txt_patch[0]
            content = load_doc_id_text(file_path)

            knowledge_base += f"# {discipline}\n"
            knowledge_base += f"## {topic}\n"
            knowledge_base += f"### {lecture_name}\n"
            knowledge_base += f"{content}\n\n"

            data_from_markdown = knowledge_base

            # Создание индекса для новой лекции
            index_path_lesson = embed_bd(data_from_markdown, folder_path)

            # Загрузка нового индекса
            db_lesson = FAISS.load_local(
                folder_path=index_path_lesson,
                embeddings=OpenAIEmbeddings(),
                allow_dangerous_deserialization=True
            )

            # Добавление новых векторов в существующую базу
            db.merge_from(db_lesson)

            # Сохранение обновленной базы данных
            db.save_local(faiss_link)

            print(f'''Новая лекция "{topic}" добавлена в базу {faiss_link}.''')
    except Exception as e:
        with output_info:
            print(f"Произошла ошибка: {e}")

# Функция для обработки нажатия кнопки
def on_add_button_click(_):
    try:
        # Получение значений из виджетов
        discipline_lesson = discipline_input.value.strip()
        topic_lesson = topic_input.value.strip()
        link_lesson = link_input.value.strip()

        # Проверка корректности ввода
        if not (discipline_lesson and topic_lesson and link_lesson):
            with output_info:
                output_info.clear_output()
                print("Ошибка: Все поля должны быть заполнены!")
            return

        # Запуск асинхронной функции через event loop
        loop = asyncio.get_event_loop()
        loop.run_until_complete(process_lecture(discipline_lesson, topic_lesson, link_lesson))
    except Exception as e:
        with output_info:
            print(f"Произошла ошибка: {e}")

# Привязка функции к кнопке
add_button.on_click(on_add_button_click)

# Группировка виджетов ввода
input_widgets = widgets.VBox([discipline_input, topic_input, link_input, add_button])

# Размещение виджетов в одну строку (слева — ввод, справа — вывод)
display(widgets.HBox([input_widgets, output_info]))

In [ ]:
#@title Форма для ответа на вопрос пользователя { display-mode: "form" }

# Виджет для вывода результата
output = widgets.Output(layout={'border': '1px solid black', 'width': '100%', 'height': '300px', 'overflow_y': 'scroll'})

# Загрузка базы знаний
with output:
    db = faiss_gdown()
    if db is not None:
        print("База знаний готова к использованию!")
        print("Задайте свой вопрос по лекциям!")
    else:
        print("Не удалось загрузить базу знаний.")
        raise ValueError("Прерываю выполнение, так как база знаний не загружена.")



# Текстовое поле для ввода запроса
text_area = widgets.Textarea(
    value='',
    placeholder='Введите ваш вопрос здесь...',
    description='Вопрос:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

display(text_area)

# Кнопка для отправки запроса
submit_button = widgets.Button(description="Отправить вопрос", button_style='success')
display(submit_button)
display(output)

# Функция обработчик нажатия на кнопку
def on_submit_button_clicked(b):
    with output:
        output.clear_output()  # Очистка предыдущего вывода
        user_question = text_area.value.strip()  # Получение текста из textarea

        if not user_question:
            print("Ошибка: Введите ваш вопрос!")
            return

        try:
            print("-" * 100)
            print(f"Обработка запроса: {user_question}")
            #message, ans = answer_index(system, user_question, model_gpt, temperature, k_chunks, db)  # Вызов функции для получения ответа
            message, ans = answer_index(
                            system=system,
                            topic=user_question,
                            search_index=db,
                            model_gpt=model_gpt,
                            temperature=temperature,
                            k_chunks=k_chunks
                        )
            print("-" * 100)
            print(message)
            output.clear_output()
            print("\n =======\nОтвет GPT:\n", ans)
        except Exception as e:
            print(f"Произошла ошибка: {e}")

# Привязка события к кнопке
submit_button.on_click(on_submit_button_clicked)

Textarea(value='', description='Вопрос:', layout=Layout(height='100px', width='100%'), placeholder='Введите ва…

Button(button_style='success', description='Отправить вопрос', style=ButtonStyle())

Output(layout=Layout(border='1px solid black', height='300px', overflow_y='scroll', width='100%'))

In [ ]:
#@title Формы для пересказа лекции - новая { display-mode: "form" }

import ipywidgets as widgets
from IPython.display import display

k_recap = 0.1

# Загрузка базы знаний
db_loaded = FAISS.load_local(
    folder_path=faiss_link,
    embeddings=OpenAIEmbeddings(),
    allow_dangerous_deserialization=True
)

# Область вывода результатов
output_ret = widgets.Output(layout={'border': '1px solid black', 'width': '70%', 'height': '300px', 'overflow_y': 'scroll'})

# Группировка виджетов ввода
with output_ret:
    if db_loaded is not None:
        print("База знаний готова к использованию!")
        print("Выберите лекцию и запустите пересказ!")
    else:
        db_loaded = faiss_gdown()
        if db_loaded is not None:
            print("База знаний готова к использованию!")
            print("Задайте свой вопрос по лекциям!")
        else:
            print("Не удалось загрузить базу знаний.")
            raise ValueError("Прерываю выполнение, так как база знаний не загружена.")

# Получение списка уникальных значений "Header 3"
doc_ids = db_loaded.index_to_docstore_id.values()
chunks_with_metadata = [db_loaded.docstore.search(doc_id) for doc_id in doc_ids]
unique_header_3_values = set(chunk.metadata.get("Header 3") for chunk in chunks_with_metadata if chunk.metadata.get("Header 3"))
sorted_header_3_values = sorted(unique_header_3_values)

# Создание виджетов для ввода данных
target_header_3 = widgets.Dropdown(
    options=sorted_header_3_values,
    description="Лекция:",
    value=None if not sorted_header_3_values else sorted_header_3_values[0],
    disabled=False
)

recap_lesson = widgets.Dropdown(
    options=['Очень короткий', 'Краткий', 'Подробный'],
    description="Длина пересказа:",
    value='Очень короткий',
    disabled=False
)

temperature_recap = widgets.FloatSlider(
    value=0.1,
    min=0.0,
    max=1.0,
    step=0.1,
    description="Температура:",
    disabled=False
)


# Функция для обработки нажатия кнопки
def on_generate_button_click(_):
    with output_ret:
        output_ret.clear_output()  # Очистка предыдущего вывода

        # Получение значений из виджетов
        selected_header_3 = target_header_3.value
        selected_recap = recap_lesson.value
        selected_temperature = temperature_recap.value
        #selected_group_chunk = group_chunk.value

        selected_group_chunk = 4
        # Проверка, что выбрано значение для Header 3
        if not selected_header_3:
            print("Ошибка: Выберите лекцию из списка!")
            return

        try:
            # Фильтрация чанков, соответствующих выбранному Header 3
            filtered_chunks = [
                (i, chunk) for i, chunk in enumerate(chunks_with_metadata)
                if chunk.metadata.get("Header 3") == selected_header_3
            ]


            # Преобразование отфильтрованных чанков в одну строку
            filtered_text = " ".join([chunk.page_content for _, chunk in filtered_chunks])

            if recap_lesson.value == 'Подробный':
                k_recap = 0.7
            elif recap_lesson.value == 'Короткий':
                k_recap = 0.5
            else:
                k_recap = 0.1

            # Подсчет количества токенов во входном тексте
            token_lesson = num_tokens_from_string(filtered_text, "cl100k_base")
            print(f"Выбрана лекция: {selected_header_3}")
            print(f"Длина пересказа: {selected_recap}")
            print(f"Температура: {selected_temperature}")

            print(f"\nКоличество токенов в исходном тексте: {token_lesson}")
            print(f"\nКоличество чанков в исходном тексте: {len (filtered_chunks)}")

            print("Количество слов:",count_tokens(filtered_text))

            # Загрузка nlp
            doc = nlp(filtered_text)
            # Формирование предложений
            sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
            print("Количество предложений", len(sentences))

            max_sentences = int(k_recap * len(sentences))

            print(k_recap)
            print('Макс Количество предложений', max_sentences)

            # Получаем наиболее релевантные предложения
            relevant_sentences = get_relevant_sentences(filtered_text, max_sentences)

            reduced_text = " ".join(relevant_sentences)
            print ('Получено релевантных предложений',len(relevant_sentences))


            #print(reduced_text)
            print(num_tokens_from_string(reduced_text, "cl100k_base"))

            # Выводим количество релевантных предложений
            print('Выбранные релевантные предложения', len(relevant_sentences))

            # Объединяем список предложений в одну строку
            relevant_text = " ".join(relevant_sentences)

            # Подсчитываем количество токенов для объединенной строки
            print('Количество токенов', num_tokens_from_string(relevant_text, "cl100k_base"))

            reduced_group = []
            rewrite_lessons = ""

            for sent in relevant_sentences:
                # Добавляем предложение в список
                reduced_group.append(sent)

                # Объединяем список в строку для подсчета токенов
                reduced_group_text = " ".join(reduced_group)

                # Проверяем количество токенов
                if num_tokens_from_string(reduced_group_text, "cl100k_base") > 2000:
                    # Передаем текст для пересказа
                    time.sleep(1)
                    # Пересказ части лекции
                    rewrite_lesson = rewrite_reduced_text(model_gpt, reduced_group_text, temperature, selected_header_3)
                    print('Количество токенов:', num_tokens_from_string(rewrite_lesson, "cl100k_base"))
                    # Суммируем пересказ
                    rewrite_lessons += rewrite_lesson
                    # Очищаем reduced_group для следующей группы предложений
                    reduced_group = []

                else:
                    continue

            # Если остались предложения в reduced_group после цикла, обрабатываем их
            if reduced_group:
                reduced_group_text = " ".join(reduced_group)
                rewrite_lesson += rewrite_reduced_text(model_gpt, reduced_group_text, temperature, selected_header_3)
                print('Последнее Количество токенов:', num_tokens_from_string(rewrite_lesson, "cl100k_base"))
                rewrite_lessons += rewrite_lesson

            output.clear_output()
            print(f"Пересказ лекции {selected_header_3}:\n{rewrite_lessons}")

            print(f'Количество токенов в пересказе: {num_tokens_from_string(rewrite_lessons, "cl100k_base")}')



        except Exception as e:
            print(f"Произошла ошибка: {e}")

# Кнопка для генерации пересказа
generate_button = widgets.Button(description="Сгенерировать пересказ", button_style='success')
generate_button.on_click(on_generate_button_click)

# Размещение виджетов в интерфейсе
input_widgets = widgets.VBox([
    target_header_3,
    recap_lesson,
    temperature_recap
    #group_chunk
])

display(widgets.HBox([input_widgets, output_ret]))
display(generate_button)

Button(button_style='success', description='Сгенерировать пересказ', style=ButtonStyle())